In [1]:
import pandas as pd
import math 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## define functions to calculate angle between two lines and rotate points around origin

# magnitude of a vector
def magvect(v):
  magnitude = np.sqrt(v[0]**2 + v[1]**2)
  return magnitude

# angle between vectors
def angle(v1,v2):
  ang = np.arccos(np.dot(v1,v2)/(magvect(v1)*magvect(v2)))
  ang = ang*180/np.pi
#  if (ang > 90):
#    ang = 180 - ang
  return ang

# def rotate_p(point, radians, origin=(0, 0)):
def rotate_p(point, radians, origin):
    # rotate point around a specified origin
    x, y = point
    ox, oy = origin

    qx = ox + math.cos(radians) * (x - ox) + math.sin(radians) * (y - oy)
    qy = oy + -math.sin(radians) * (x - ox) + math.cos(radians) * (y - oy)
    
    p_rot = [qx,qy]
    
    return p_rot

In [3]:
## import data
# excel file to calculate OA + width length measured in illustrator
input_path_s1 = '/Users/Suppl4_element_geometries/S1_AspectRatio_Opening_Angle.xlsx'
input_path_s2 = '/Users/Suppl4_element_geometries/S2_AspectRatio_Opening_Angle.xlsx'

# initialise empty arrays for sill 1 and sill 2
xl1   = pd.ExcelFile(input_path_s1)
res1    = len(xl1.sheet_names) # number of sheets
xl2   = pd.ExcelFile(input_path_s2)
res2    = len(xl2.sheet_names) # number of sheets

sill1  = np.zeros((res1,5)) # type, width, length, AR, mean OA
sill2  = np.zeros((res2,5)) # type, width, length, AR, mean OA

In [4]:
# calculate mean angle aplha and create matrix with data for plots

## sill 1
s1t1A = [0,1,2,3,6,7,8,10,11,14,16,18,19,21,22,23]
s1t1B = [5,12,17,20]
s1t2 = [4,9,13,15]  

sill1[s1t1A,0] = 11 #1A
sill1[s1t1B,0] = 12 #1B
sill1[s1t2,0]  = 2  #2

for k in range(res1): # sill 1
    data = pd.read_excel(input_path_s1, sheet_name=k) # import excel sheet
    sill1[k,1] = data['width'].values[0] # lobe width
    sill1[k,2] = data['length'].values[0] # lobe length
    sill1[k,3] = sill1[k,1]/sill1[k,2] # AR

    
    ## mean opening angle
    ID = data['line_ID'].values # line ID
    vertID = data['vertex_ind'].values # vertex index
    x = data['X'].values # x coordinate
    y = data['Y'].values # y coordinate
    x = np.round(x) # round values
    y = np.round(y) # round values
    
    # separate data
    ind0 = np.where(ID==0) # angle bisector
    ind1 = np.where(ID==1) # LHS
    ind2 = np.where(ID==2) # RHS

    x0 = np.round(data['X'].values[ind0])
    x1 = np.round(data['X'].values[ind1])
    x2 = np.round(data['X'].values[ind2])

    y0 = np.round(data['Y'].values[ind0])
    y1 = np.round(data['Y'].values[ind1])
    y2 = np.round(data['Y'].values[ind2])

    
    ## calculate angle between flow direction (ind0) and vertical
    # define vector
    vec1 = [x0[1]-x0[0],y0[1]-y0[0]] # flow direction
    vec2 = [0,10] # vertical

    # angle
    rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
    alpha = np.rad2deg(rad_a)
    alpha

    # rotate (flow direction)
    rot_flow_dir = rotate_p([x0[1],y0[1]], -rad_a, [x0[0], y0[0]])

    # rotate (LHS)
    size_ind1 = np.size(ind1)
    rot_LHS = np.zeros((size_ind1,2))

    for i in range(np.size(ind1)):
        rot_LHS[i,:] = rotate_p([x1[i],y1[i]], -rad_a, [x0[0], y0[0]])

    # rotate (RHS)
    size_ind2 = np.size(ind2)
    rot_RHS = np.zeros((size_ind2,2))

    for i in range(np.size(ind2)):
        rot_RHS[i,:] = rotate_p([x2[i],y2[i]], -rad_a, [x0[0], y0[0]])



    ## calculate angle of RHS (or LHS) to magma flow direction
    # LHS
    size_ind1 = np.size(ind1)
    alpha = np.zeros(size_ind1-1)
    for i in range(1, np.size(ind1)):
        vec1 = rot_LHS[i]-rot_LHS[i-1]
        vec2 = [0,10]

        rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
        alpha[i-1] = -np.rad2deg(rad_a)

    # RHS
    size_ind2 = np.size(ind2)
    beta = np.zeros(size_ind2-1)
    for i in range(1, np.size(ind2)):
        vec1 = rot_RHS[i]-rot_RHS[i-1]
        vec2 = [0,10]

        rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
        beta[i-1] = np.rad2deg(rad_a)  
    
    
    
    ## create matrix with calculated data
    # [y, alpha LHS, beta RHS, OA(LHS+RHS), length]

    # initialise arrays
    size_ind1 = np.size(ind1) # number of points LHS
    size_ind2 = np.size(ind2) # number of points RHS
    data_plot = np.zeros((size_ind1+size_ind2,6))

    # xcoord
    data_plot[0:size_ind1, 0] = np.round(rot_LHS[:,0])
    data_plot[size_ind1:(size_ind1+size_ind2),0] = np.round(rot_RHS[:,0])

    # ycoord
    data_plot[0:size_ind1, 1] = np.round(rot_LHS[:,1])
    data_plot[size_ind1:(size_ind1+size_ind2),1] = np.round(rot_RHS[:,1])

    # opening angle
    data_plot[0, 2] = alpha[0] # LHS, first calculated angle for first coordinate
    data_plot[1:size_ind1, 2] = alpha # LHS
    data_plot[size_ind1, 3] = beta[0] # RHS, first calculated angle for first coordinate
    data_plot[(size_ind1+1):(size_ind1+size_ind2), 3] = beta # RHS

    # length
    min_len = np.amin(data_plot[:,1])
    data_plot[0:size_ind1, 5] = y1-min_len # LHS
    data_plot[size_ind1:(size_ind1+size_ind2),5] = y2-min_len # RHS

    # list with 1m steps in length
    y_min = np.amin(data_plot[:,1])
    y_max = np.amax(data_plot[:,1])
    y_len = y_max-y_min
    data_plot1 = np.zeros((int(y_len+1),5)) # [y, alpha, beta, OA, length]
    data_plot1[:] = np.nan
    data_plot1[:,0] = np.arange(np.amin(data_plot[:,1]), np.amax(data_plot[:,1])+1)

    # add opening angles LHS
    for i in range (size_ind1):

        if i == size_ind1-1:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]))
            data_plot1[ind_temp,1] = data_plot[i,2] # alpha
        else:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]) & (data_plot1[:,0]<data_plot[i+1,1]))
            data_plot1[ind_temp,1] = data_plot[i+1,2] # alpha


    # add opening angles RHS
    for i in range (size_ind1,size_ind1+size_ind2):    

        if i == size_ind1+size_ind2-1:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[-1,1]) & (data_plot1[:,0]<=data_plot1[-1,0]))
            data_plot1[ind_temp,2] = data_plot[i,3] # beta
        else:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]) & (data_plot1[:,0]<=data_plot[i+1,1]))
            data_plot1[ind_temp,2] = data_plot[i+1,3] # beta


    # Add alpha (or) beta at the beginning (and/or end) in case of zeros
    # beginning
    
    ind_temp = np.where(np.isnan(data_plot1[:,1])) # LHS
    data_plot1[ind_temp,1] = alpha[0]
    
    ind_temp = np.where(np.isnan(data_plot1[:,2])) # RHS
    data_plot1[ind_temp,2] = beta[0]


    # calculate total opening angle (LHS + RHS)
    data_plot1[:,3] = data_plot1[:,1] + data_plot1[:,2] # OA

    # calculate length
    for i in range (len(data_plot1)):
        data_plot1[i,4] = data_plot1[i,0]-data_plot1[0,0]
        
        
    ## average / mean opening angle (weighted from length)
    # create input array to plot mean OA vs length [mean OA, total length]
    uni = np.unique(data_plot1[:,3])
    temp_array = np.zeros((len(uni),3))
    meanOA_temp = np.zeros([1,2])

    for i in range(len(uni)):
        ind_temp = np.where((data_plot1[:,3] == uni[i]))
        weight = (np.size(np.where((data_plot1[:,3] == uni[i])),1)) / np.amax(data_plot1[:,4])
        weighted_OA = weight * uni[i]
        temp_array[i,0] = uni[i]
        temp_array[i,1] = weight
        temp_array[i,2] = weighted_OA

    sill1[k,4] = np.sum(temp_array[:,2]) # weighted mean OA
    

## sill 2
s2t1A = [1,2,3,4,6,9,10,16,21,22,23,25,27]
s2t1B = [0,12,14,24,26]
s2t2  = [5,7,8,11,13,15,17,18,19,20]

sill2[s2t1A,0] = 11 #1A
sill2[s2t1B,0] = 12 #1B
sill2[s2t2,0]  = 2  #2

for k in range(res2): # sill 2
    data = pd.read_excel(input_path_s2, sheet_name=k) # import excel sheet
    sill2[k,1] = data['width'].values[0] # lobe width
    sill2[k,2] = data['length'].values[0] # lobe length
    sill2[k,3] = sill2[k,1]/sill2[k,2] # AR

    
    ## mean opening angle
    ID = data['line_ID'].values # line ID
    vertID = data['vertex_ind'].values # vertex index
    x = data['X'].values # x coordinate
    y = data['Y'].values # y coordinate
    x = np.round(x) # round values
    y = np.round(y) # round values
    
    # separate data
    ind0 = np.where(ID==0) # angle bisector
    ind1 = np.where(ID==1) # LHS
    ind2 = np.where(ID==2) # RHS

    x0 = np.round(data['X'].values[ind0])
    x1 = np.round(data['X'].values[ind1])
    x2 = np.round(data['X'].values[ind2])

    y0 = np.round(data['Y'].values[ind0])
    y1 = np.round(data['Y'].values[ind1])
    y2 = np.round(data['Y'].values[ind2])

    
    ## calculate angle between flow direction (ind0) and vertical
    # define vector
    vec1 = [x0[1]-x0[0],y0[1]-y0[0]] # flow direction
    vec2 = [0,10] # vertical

    # angle
    rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
    alpha = np.rad2deg(rad_a)
    alpha

    # rotate (flow direction)
    rot_flow_dir = rotate_p([x0[1],y0[1]], -rad_a, [x0[0], y0[0]])

    # rotate (LHS)
    size_ind1 = np.size(ind1)
    rot_LHS = np.zeros((size_ind1,2))

    for i in range(np.size(ind1)):
        rot_LHS[i,:] = rotate_p([x1[i],y1[i]], -rad_a, [x0[0], y0[0]])

    # rotate (RHS)
    size_ind2 = np.size(ind2)
    rot_RHS = np.zeros((size_ind2,2))

    for i in range(np.size(ind2)):
        rot_RHS[i,:] = rotate_p([x2[i],y2[i]], -rad_a, [x0[0], y0[0]])



    ## calculate angle of RHS (or LHS) to magma flow direction
    # LHS
    size_ind1 = np.size(ind1)
    alpha = np.zeros(size_ind1-1)
    for i in range(1, np.size(ind1)):
        vec1 = rot_LHS[i]-rot_LHS[i-1]
        vec2 = [0,10]

        rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
        alpha[i-1] = -np.rad2deg(rad_a)

    # RHS
    size_ind2 = np.size(ind2)
    beta = np.zeros(size_ind2-1)
    for i in range(1, np.size(ind2)):
        vec1 = rot_RHS[i]-rot_RHS[i-1]
        vec2 = [0,10]

        rad_a = math.atan2(vec1[0]*vec2[1]-vec1[1]*vec2[0],vec1[0]*vec2[0]+vec1[1]*vec2[1]) 
        beta[i-1] = np.rad2deg(rad_a)  
    
    
    
    ## create matrix with calculated data
    # [y, alpha LHS, beta RHS, OA(LHS+RHS), length]

    # initialise arrays
    size_ind1 = np.size(ind1) # number of points LHS
    size_ind2 = np.size(ind2) # number of points RHS
    data_plot = np.zeros((size_ind1+size_ind2,6))

    # xcoord
    data_plot[0:size_ind1, 0] = np.round(rot_LHS[:,0])
    data_plot[size_ind1:(size_ind1+size_ind2),0] = np.round(rot_RHS[:,0])

    # ycoord
    data_plot[0:size_ind1, 1] = np.round(rot_LHS[:,1])
    data_plot[size_ind1:(size_ind1+size_ind2),1] = np.round(rot_RHS[:,1])

    # opening angle
    data_plot[0, 2] = alpha[0] # LHS, first calculated angle for first coordinate
    data_plot[1:size_ind1, 2] = alpha # LHS
    data_plot[size_ind1, 3] = beta[0] # RHS, first calculated angle for first coordinate
    data_plot[(size_ind1+1):(size_ind1+size_ind2), 3] = beta # RHS

    # length
    min_len = np.amin(data_plot[:,1])
    data_plot[0:size_ind1, 5] = y1-min_len # LHS
    data_plot[size_ind1:(size_ind1+size_ind2),5] = y2-min_len # RHS

    # list with 1m steps in length
    y_min = np.amin(data_plot[:,1])
    y_max = np.amax(data_plot[:,1])
    y_len = y_max-y_min
    data_plot1 = np.zeros((int(y_len+1),5)) # [y, alpha, beta, OA, length]
    data_plot1[:] = np.nan
    data_plot1[:,0] = np.arange(np.amin(data_plot[:,1]), np.amax(data_plot[:,1])+1)

    # add opening angles LHS
    for i in range (size_ind1):

        if i == size_ind1-1:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]))
            data_plot1[ind_temp,1] = data_plot[i,2] # alpha
        else:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]) & (data_plot1[:,0]<data_plot[i+1,1]))
            data_plot1[ind_temp,1] = data_plot[i+1,2] # alpha


    # add opening angles RHS
    for i in range (size_ind1,size_ind1+size_ind2):    

        if i == size_ind1+size_ind2-1:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[-1,1]) & (data_plot1[:,0]<=data_plot1[-1,0]))
            data_plot1[ind_temp,2] = data_plot[i,3] # beta
        else:
            ind_temp = np.where((data_plot1[:,0]>=data_plot[i,1]) & (data_plot1[:,0]<=data_plot[i+1,1]))
            data_plot1[ind_temp,2] = data_plot[i+1,3] # beta


    # Add alpha (or) beta at the beginning (and/or end) in case of zeros
    # beginning
    
    ind_temp = np.where(np.isnan(data_plot1[:,1])) # LHS
    data_plot1[ind_temp,1] = alpha[0]
    
    ind_temp = np.where(np.isnan(data_plot1[:,2])) # RHS
    data_plot1[ind_temp,2] = beta[0]


    # calculate total opening angle (LHS + RHS)
    data_plot1[:,3] = data_plot1[:,1] + data_plot1[:,2] # OA

    # calculate length
    for i in range (len(data_plot1)):
        data_plot1[i,4] = data_plot1[i,0]-data_plot1[0,0]
        
        
    ## average / mean opening angle (weighted from length)
    # create input array to plot mean OA vs length [mean OA, total length]
    uni = np.unique(data_plot1[:,3])
    temp_array = np.zeros((len(uni),3))
    meanOA_temp = np.zeros([1,2])

    for i in range(len(uni)):
        ind_temp = np.where((data_plot1[:,3] == uni[i]))
        weight = (np.size(np.where((data_plot1[:,3] == uni[i])),1)) / np.amax(data_plot1[:,4])
        weighted_OA = weight * uni[i]
        temp_array[i,0] = uni[i]
        temp_array[i,1] = weight
        temp_array[i,2] = weighted_OA

    sill2[k,4] = np.sum(temp_array[:,2]) # weighted mean OA    

In [5]:
## array to pandas data frame to *.csv
# S1
s1_df = pd.DataFrame(sill1)
s1_df = s1_df.rename(columns={0:'Element_type', 1:'Width', 2:'Length', 3:'AR', 4:'alpha'})
s1_df.insert(0,'Sill', 'S1')
s1_df=s1_df.replace({'Element_type':11},'1A')
s1_df=s1_df.replace({'Element_type':12},'1B')

# S2
s2_df = pd.DataFrame(sill2)
s2_df = s2_df.rename(columns={0:'Element_type', 1:'Width', 2:'Length', 3:'AR', 4:'alpha'})
s2_df.insert(0,'Sill', 'S2')
s2_df=s2_df.replace({'Element_type':11},'1A')
s2_df=s2_df.replace({'Element_type':12},'1B')
s2_df

s1s2_df = s1_df.append(s2_df, ignore_index=True)

s1s2_df.to_excel('/Users/Suppl4_element_geometries/4-AspectRatio_alpha.xlsx')